## Implementação Prática
A _Cia Ótica Gallão_ pretende utilizar os canteiros do mapa rodoviário abaixo para passar o novo cabeamento de fibra ótica entre essas cidades, com base na origem e destino que você informar:

<div align="center">
    <img src="./src/img/mapaCidades.png" alt="Mapa Rodoviário" />
</div> 

In [11]:
# Importação das bibliotecas necessárias para exibição do diagrama
import base64 
from IPython.display import Image 

# Implementação do algoritmo de Kruskal 
class Grafo:
    # Inicializa o objeto Grafo com o grafo passado como parâmetro
    def __init__(self, grafo):
        self.grafo = grafo
        self.arestas = []

        for cidade, conexoes in grafo.items():
            for conexao in conexoes:
                cidade_destino, peso = conexao
                self.arestas.append((peso, cidade, cidade_destino))

        # Ordena as arestas pelo peso (primeiro elemento)
        self.arestas.sort()  

    # Encontra o predecessor (pai) de um vértice
    def encontra(self, predecessor, vertice):
        if predecessor[vertice] == vertice:
            return vertice
        
        return self.encontra(predecessor, predecessor[vertice])

    # Realiza a união de dois vértices 
    def uniao(self, predecessor, nivel, vertice_01, vertice_02):
        raiz_01 = self.encontra(predecessor, vertice_01)
        raiz_02 = self.encontra(predecessor, vertice_02)

        # Verifica se os vértices possuem a mesma raiz
        if nivel[raiz_01] < nivel[raiz_02]:         
            predecessor[raiz_01] = raiz_02          
        elif nivel[raiz_01] > nivel[raiz_02]:       # Se o nível do vértice 01 for maior que o nível do vértice 02 
            predecessor[raiz_02] = raiz_01          # o vértice 02 se torna o pai do vértice 01
        else:
            predecessor[raiz_02] = raiz_01          
            nivel[raiz_01] += 1                     # Incrementa o nível do vértice 01

    # Algoritmo de Kruskal 
    def algoritmoKruskal(self):
        arvore = []         # Armazena as arestas da árvore mínima
        predecessor = {}    # Armazena o pai de cada cidade
        nivel = {}          # Armazena o nível (geração) de cada cidade

        # Inicialização: cada cidade é seu próprio pai
        for cidade in self.grafo.keys():
            predecessor[cidade] = cidade
            nivel[cidade] = 0

        # Percorre todas as arestas e aplica a lógica de Kruskal
        for aresta in self.arestas:
            peso, cidade_01, cidade_02 = aresta

            # Encontra a raiz de cada cidade
            raiz_01 = self.encontra(predecessor, cidade_01)     
            raiz_02 = self.encontra(predecessor, cidade_02)     

            # Se as cidades possuem raízes diferentes
            if raiz_01 != raiz_02:
                arvore.append((cidade_01, cidade_02, peso))             # Adiciona a aresta à árvore mínima
                self.uniao(predecessor, nivel, raiz_01, raiz_02)        # Realiza a união dos vértices

        return arvore

# Exibe o diagrama Mermaid
def exibe_mermaid_flowchart(grafo):
    graphbytes = grafo.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    display(Image(url="https://mermaid.ink/img/" + base64_string))

# Exibe a árvore mínima completa
def exibe_arvore_minima(arvore, nome_das_cidades):
    print("Árvore de peso mínimo completa:")
    
    flowchart = "flowchart TD\n"
    for cidade_01, cidade_02, peso in arvore:
        flowchart += f"    {nome_das_cidades[cidade_01]}([{cidade_01}]) --- |{peso}| {nome_das_cidades[cidade_02]}([{cidade_02}])\n"

    return flowchart

# Constrói o grafo da árvore mínima
def constroi_arvore(arvore):
    grafo = {}

    for vertice_01, vertice_02, peso in arvore:
        if vertice_01 not in grafo:
            grafo[vertice_01] = []

        if vertice_02 not in grafo:
            grafo[vertice_02] = []

        # Adiciona a conexão entre os vértices
        grafo[vertice_01].append((vertice_02, peso))
        grafo[vertice_02].append((vertice_01, peso))
    return grafo

# Busca em profundidade para encontrar o caminho entre a origem e o destino
def dfs_caminho(grafo, origem, destino, visitado=None, caminho=None, distancia=0):
    if visitado is None:
        visitado = set()
    if caminho is None:
        caminho = []

    visitado.add(origem)
    caminho.append((origem, distancia))

    if origem == destino:
        return caminho

    for vizinho, peso in grafo.get(origem, []):
        if vizinho not in visitado:
            resultado = dfs_caminho(grafo, vizinho, destino, visitado, caminho, peso)
            if resultado:
                return resultado

    caminho.pop()
    return None

# Calcula a distância total percorrida no caminho
def calcular_distancia(caminho):
    distancia_total = 0
    
    for i in range(1, len(caminho)):
        distancia_total += caminho[i][1]

    return distancia_total

# Grafo do mapa rodoviário
grafo = {
    'Piracicaba': [['Americana', 30], ['Capivari', 32], ['Tietê', 35]],
    'Americana': [['Sumaré', 18], ['Paulínea', 22], ['Piracicaba',30]],
    'Capivari': [['Monte Mor', 15], ['Salto', 25], ['Tietê', 30], ['Piracicaba', 32]],
    'Tietê': [['Tatuí', 25], ['Capivari', 30], ['Porto Feliz', 30], ['Piracicaba', 35]],
    'Paulínea': [['Americana', 22], ['Campinas', 25]],
    'Sumaré': [['Americana', 18], ['Campinas', 23]],
    'Monte Mor': [['Capivari', 15], ['Campinas', 22]],
    'Indaiatuba': [['Campinas', 20], ['Salto', 20]],
    'Salto': [['Itú', 10], ['Indaiatuba', 20], ['Capivari', 25]],
    'Porto Feliz': [['Itú', 12], ['Boituva', 12], ['Tietê', 30]],
    'Tatuí': [['Boituva', 17], ['Tietê', 25]],
    'Campinas': [['Indaiatuba', 20], ['Monte Mor', 22], ['Sumaré', 23], ['Paulínea', 25]],
    'Itú': [['Sorocaba', 8], ['Salto', 10], ['Porto Feliz', 12]],
    'Sorocaba': [['Itú', 8], ['Boituva', 23]],
    'Boituva': [['Porto Feliz', 12], ['Tatuí', 17], ['Sorocaba', 23]]
}

# Associação entre os nomes das cidades e os vértices do grafo (para exibição)
nome_das_cidades = {'Piracicaba': 'Piracicaba', 'Americana': 'Americana', 'Sumaré': 'Sumare', 
                 'Campinas': 'Campinas', 'Monte Mor': 'MonteMor', 'Indaiatuba': 'Indaiatuba',
                 'Salto': 'Salto', 'Capivari': 'Capivari', 'Tietê': 'Tiete', 'Paulínea': 'Paulinea',
                 'Porto Feliz': 'PortoFeliz', 'Tatuí': 'Tatui', 'Itú': 'Itu', 'Sorocaba': 'Sorocaba',
                 'Boituva': 'Boituva'}

# Interação com o usuário
origem = input("Digite a cidade de origem: ")
destino = input("Digite a cidade de destino: ")

# Execução do algoritmo
g = Grafo(grafo)        # Cria o objeto Grafo 

arvore_minima = g.algoritmoKruskal()                # Executa o algoritmo de Kruskal para obter a árvore mínima
grafo_arvore = constroi_arvore(arvore_minima)       # Constrói o grafo da árvore mínima

# Encontra o caminho da origem até o destino na árvore mínima
caminho = dfs_caminho(grafo_arvore, origem, destino)

## Exibição do resultado
if caminho:
    print(f"\nO menor caminho identificado entre as cidades de {origem} e {destino} é composto pelas seguintes cidades:")

    flowchart = "flowchart LR\n"
    for cidade, distancia in caminho:   
        if distancia == 0:
            flowchart += f"    {nome_das_cidades[cidade]}([{cidade}])\n"
        else:
            flowchart += f"    -- {distancia} --> {nome_das_cidades[cidade]}([{cidade}]) \n"

    # Exibe o caminho encontrado entre as cidades como um diagrama Mermaid
    exibe_mermaid_flowchart(flowchart) 

    # Exibe a distância total a ser percorrida 
    distancia_total = calcular_distancia(caminho)
    print(f"A distância total a ser percorrida entre cada cidade é de: {distancia_total}km.\n")
else:
    print(f"Não foi possível encontrar um caminho de {origem} até {destino}.")

# Exibe a árvore mínima completa
exibe_mermaid_flowchart(exibe_arvore_minima(arvore_minima, nome_das_cidades))


O menor caminho identificado entre as cidades de Salto e Boituva é composto pelas seguintes cidades:


A distância total a ser percorrida entre cada cidade é de: 34km.

Árvore de peso mínimo completa:
